In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial.distance import pdist, squareform

In [60]:
names = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven',
         'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen'] #this creates dummy names for the formations
number_of_layers = 7 #this is the number of tops you want in your training data
smallest = -5
largest = 7
step = 0.1
no_of_neighbors = 20

In [61]:
df =  pd.DataFrame()
locations = pd.DataFrame()
elevation_random = sorted(np.random.uniform(smallest, largest, number_of_layers-1))
for j in np.arange(smallest, largest, step):
    rolling = pd.DataFrame()
    for i in range(len(names[0:number_of_layers-1])):
        basement = 10*np.sin(1-np.arange(0,40,0.1)/15.28)
        elevation = np.full(400, j)
        topbasement = np.where(basement > elevation, elevation, basement)
        rolling['zero'] = topbasement
        layer_elevation = 10*np.sin(1-np.arange(0,40,0.1)/15.28)+elevation_random[i]
        layer_elevation = np.where(layer_elevation > elevation, elevation, layer_elevation)
        rolling[names[i]] = layer_elevation
    x = np.arange(0,40,0.1)
    y = np.random.randint(0,10,len(x))
    if j%0.2 >0.1:
        rolling['ex'] = x*np.cos(-j/2) - y*np.sin(-j/2)
        rolling['ey'] = y*np.cos(-j/2) - x*np.sin(-j/2)
    else:
        rolling['ex'] = x*np.cos(j/2) - y*np.sin(j/2)
        rolling['ey'] = y*np.cos(j/2) - x*np.sin(j/2)
    for k in range(100):
        rolling.iloc[np.random.randint(0,399), np.random.randint(0,number_of_layers-1)] = 0
    hood = squareform(pdist(rolling.iloc[:, -2:]))
    neighbors = [] 
    for i in enumerate(hood.argsort()[0:,1:no_of_neighbors+1]):
        selected = rolling.iloc[hood.argsort()[i[0],1:no_of_neighbors+1], 0:-2].stack().to_frame().T
        selected.columns = selected.columns.droplevel()
        neighbors.append(selected)
    frame = pd.concat(neighbors, sort=False)
    frame.index = range(len(frame))
    neighborhood = pd.concat([rolling.iloc[:,:-2], frame], axis=1)
    thicknesses  = neighborhood.diff(axis=1)
    thicknesses[thicknesses < 0] = 0
    thicknesses.drop(columns='zero', inplace=True)
    locations = pd.concat((locations, rolling.iloc[:,-2:]))
    df = pd.concat((df, thicknesses))
logged = df.apply(np.log) #take the log of thicknesses for feature engineering
powered = df.apply(lambda x: x**10) #calculates the power values of thickness for another feature
at = pd.concat([df, logged, powered, locations], axis=1, join_axes=[df.index]
                        ).dropna().replace(-np.inf, 0) 


In [62]:
at.shape

(48000, 380)

In [63]:
#NORMALIZING THE DATA
#normalize the data from 0 to 1
normalized_dfa = (at-at.min())/(at.max()-at.min()).replace(0,0.00001)
normalized_locations = (locations-locations.min())/(locations.max()-locations.min())
x = normalized_locations.ex.values
y = normalized_locations.ey.values
normalized_dfa['ex'] = x
normalized_dfa['ey'] = y

In [ ]:
df_onlap =  pd.DataFrame()
locations = pd.DataFrame()
for j in np.arange(smallest, largest, step):
    rolling = pd.DataFrame()
    for i in range(len(names[0:number_of_layers-1])):
        basement = 10*np.sin(1-np.arange(0,40,0.1)/15.28)
        elevation = np.full(400, j)
        topbasement = np.where(basement > elevation, elevation, basement)
        rolling['zero'] = topbasement
        strat_elevation = np.full(400, elevation_random[i])
        onlap = np.where(strat_elevation > basement, strat_elevation, basement)
        layer_elevation = np.where(onlap > elevation, elevation, onlap)
        rolling[names[i]] = layer_elevation
    x = np.arange(0,40,0.1)
    y = np.random.randint(0,10,len(x))
    if j%0.2 >0.1:
        rolling['ex'] = x*np.cos(-j/2) - y*np.sin(-j/2)
        rolling['ey'] = y*np.cos(-j/2) - x*np.sin(-j/2)
    else:
        rolling['ex'] = x*np.cos(j/2) - y*np.sin(j/2)
        rolling['ey'] = y*np.cos(j/2) - x*np.sin(j/2)
    for k in range(100):
        rolling.iloc[np.random.randint(0,399), np.random.randint(0,number_of_layers-1)] = 0
    hood = squareform(pdist(rolling.iloc[:, -2:]))
    neighbors = [] 
    for i in enumerate(hood.argsort()[0:,1:no_of_neighbors+1]):
        selected = rolling.iloc[hood.argsort()[i[0],1:no_of_neighbors+1], 0:-2].stack().to_frame().T
        selected.columns = selected.columns.droplevel()
        neighbors.append(selected)
    frame = pd.concat(neighbors, sort=False)
    frame.index = range(len(frame))
    neighborhood = pd.concat([rolling.iloc[:,:-2], frame], axis=1)
    thicknesses  = neighborhood.diff(axis=1)
    thicknesses[thicknesses < 0] = 0
    thicknesses.drop(columns='zero', inplace=True)
    locations = pd.concat((locations, rolling.iloc[:,-2:]))
    df_onlap = pd.concat((df_onlap, thicknesses))
onlaplogged = df_onlap.apply(np.log) #take the log of thicknesses for feature engineering
onlappowered = df_onlap.apply(lambda x: x**10) #calculates the power values of thickness for another feature
ot = pd.concat([df_onlap, onlaplogged, onlappowered, locations], axis=1, join_axes=[df_onlap.index]
                        ).dropna().replace(-np.inf, 0) 

In [ ]:
#NORMALIZING THE DATA
#normalize the data from 0 to 1
normalized_dfo = (ot-ot.min())/(ot.max()-ot.min()).replace(0,0.00001)
normalized_locations = (locations-locations.min())/(locations.max()-locations.min())
x = normalized_locations.ex.values
y = normalized_locations.ey.values
normalized_dfo['ex'] = x
normalized_dfo['ey'] = y

In [ ]:
df_horizontal =  pd.DataFrame()
locations = pd.DataFrame()
for j in np.arange(smallest, largest, step):
    rolling = pd.DataFrame()
    for i in range(len(names[0:number_of_layers-1])):
        basement = np.full(400, 0)-np.random.rand(400)/100
        elevation = np.full(400, j)
        topbasement = np.where(basement > elevation, elevation, basement)
        rolling['zero'] = topbasement
        strat_elevation = np.full(400, elevation_random[i])
        layer_elevation = np.where(strat_elevation > elevation, elevation, strat_elevation)
        rolling[names[i]] = layer_elevation
    x = np.arange(0,40,0.1)
    y = np.random.randint(0,10,len(x))
    if j%0.2 >0.1:
        rolling['ex'] = x*np.cos(-j/2) - y*np.sin(-j/2)
        rolling['ey'] = y*np.cos(-j/2) - x*np.sin(-j/2)
    else:
        rolling['ex'] = x*np.cos(j/2) - y*np.sin(j/2)
        rolling['ey'] = y*np.cos(j/2) - x*np.sin(j/2)
    for k in range(100):
        rolling.iloc[np.random.randint(0,399), np.random.randint(0,number_of_layers-1)] = 0
    hood = squareform(pdist(rolling.iloc[:, -2:]))
    neighbors = [] 
    for i in enumerate(hood.argsort()[0:,1:no_of_neighbors+1]):
        selected = rolling.iloc[hood.argsort()[i[0],1:no_of_neighbors+1], 0:-2].stack().to_frame().T
        selected.columns = selected.columns.droplevel()
        neighbors.append(selected)
    frame = pd.concat(neighbors, sort=False)
    frame.index = range(len(frame))
    neighborhood = pd.concat([rolling.iloc[:,:-2], frame], axis=1)
    thicknesses  = neighborhood.diff(axis=1)
    thicknesses[thicknesses < 0] = 0
    thicknesses.drop(columns='zero', inplace=True)
    locations = pd.concat((locations, rolling.iloc[:,-2:]))
    df_horizontal = pd.concat((df_horizontal, thicknesses))
horizlogged = df_horizontal.apply(np.log) #take the log of thicknesses for feature engineering
horizpowered = df_horizontal.apply(lambda x: x**10) #calculates the power values of thickness for another feature
hs = pd.concat([df_horizontal, horizlogged, horizpowered, locations], axis=1, join_axes=[df_horizontal.index]
                        ).dropna().replace(-np.inf, 0) 

In [ ]:
selected

In [ ]:
#NORMALIZING THE DATA
#normalize the data from 0 to 1
normalized_dfh = (hs-hs.min())/(hs.max()-hs.min()).replace(0,0.00001)
normalized_locations = (locations-locations.min())/(locations.max()-locations.min())
x = normalized_locations.ex.values
y = normalized_locations.ey.values
normalized_dfh['ex'] = x
normalized_dfh['ey'] = y

In [ ]:
#now assign classes to the datasets, 1 is onlap, 0 is angular unconformity
normalized_dfa['class'] = 0
normalized_dfo['class'] = 1
normalized_dfh['class'] = 2

In [ ]:
normalized_dfa.shape

In [ ]:
normalized_dfo.shape

In [ ]:
normalized_dfh.shape

In [ ]:
dataset = pd.concat((normalized_dfa, normalized_dfo, normalized_dfh))

In [ ]:
from sklearn.model_selection import train_test_split
#next let's split our toy data into training and test sets, choose how much with test_size of the data becomes the test set
X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[0:, 0:-1].values, dataset.iloc[0:,-1].values, 
                                                    test_size=0.1, random_state=86)

In [ ]:
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train) #this converts the target variable to one-hot encoding
y_test_encoded = to_categorical(y_test) #same with the test data conversion to one-hot encoding

In [ ]:
#Import keras and some layers, we are going to build a network with two dense layers and a dropout layer
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
#this is the network, we initialize with a dense layer and a relu activation
model = Sequential()
model.add(Dense(100, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.1)) #dropout to avoid overfitting
model.add(Dense(1000, kernel_initializer='random_uniform', activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(3, kernel_initializer='random_uniform', activation='softmax')) #and another dense layer with sigmoid activation

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['categorical_accuracy']) 
#builds the model, with categorical crossentropy for our loss function, optimizing using nadam, and using categorical accuracy
#as our accuracy metric

In [ ]:
from keras.callbacks import TensorBoard
tbCallBack = TensorBoard(log_dir='./Graph', histogram_freq=10, write_graph=True, write_images=True)

In [ ]:
model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, verbose=1, validation_split=0.2) #, callbacks=[tbCallBack]) #Fitting the model for 60 epochs and batch size of 32

In [ ]:
score=model.evaluate(X_test, y_test_encoded) #tests the model on the test data
score

In [ ]:
preds = model.predict(X_test) #creates predictions on the test data that we can use
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_test, np.argmax(preds, axis=1), normalize=True) #let's visualize the predictions

# Real World Data

In [ ]:
#let's import some real world data from southwestern Wyoming
#this is from the Ro`ck Springs Uplift in Wyoming and has been interpreted as an angular unconformity
tops_api = pd.read_csv(r'D:\jupyter\EarlyWSGS\ftunion.csv').fillna(0) #this file is available in the unconformity or onlap folder in the repo
#tops = tops_api[['Kl', 'Kll', 'Klz', 'Kfh']]
tops = tops_api[['Kfh', 'Klz', 'Kll', 'Kl', 'Tfc', 'Tfob', 'Tfu', 'x', 'y']]
locations = pd.DataFrame()

In [ ]:
hood = squareform(pdist(tops.iloc[:, -2:]))
neighbors = []
for i in enumerate(hood.argsort()[0:,1:no_of_neighbors+1]):
    selected = tops.iloc[hood.argsort()[i[0],1:no_of_neighbors+1], 0:-2].stack().to_frame().T
    selected.columns = selected.columns.droplevel()
    neighbors.append(selected)
frame = pd.concat(neighbors, sort=False)
frame.index = range(len(frame))
neighborhood = pd.concat([tops.iloc[:,:-2], frame], axis=1)
thicknesses  = neighborhood.diff(axis=1)*-1
thicknesses[thicknesses < 0] = 0
thicknesses.drop(columns='Kfh', inplace=True)
thicknesses[thicknesses < 0] = 0
thicknesses[thicknesses>3000] = 0
locations = tops[['x','y']]
real_world_log = thicknesses.apply(np.log) #take the log of thicknesses for feature engineering
real_world_pow = thicknesses.apply(lambda x: x**10) #calculates the power values of thickness for another feature
rw = pd.concat([thicknesses, real_world_log, real_world_pow, locations], axis=1, join_axes=[thicknesses.index]
                        ).dropna().replace(-np.inf, 0)

In [ ]:
normalized_rw=(rw-rw.min())/(rw.max()-rw.min()).replace(0,0.00001) #normalize the data from 0 to 1
real_data = normalized_rw.values

In [ ]:
well_preds = model.predict(real_data) #make some predictions

In [ ]:
predictions = np.argmax(well_preds, axis=1) #convert them to our binary classification
plt.hist(predictions, label = ['Angular Unconformity', 'Onlap']) #visualize the predictions as a histogram, we are expecting all the wells to be classified as '0'


In [ ]:
tops_api['predictions'] = predictions
well_locs = pd.read_csv(r'D:\jupyter\EarlyWSGS\well_locations.csv', encoding = "ISO-8859-1")
merged = pd.merge(tops_api, well_locs, on='API')
plt.scatter(merged[merged['predictions']==0].LON, merged[merged['predictions']==0].LAT, label='Angular Unconformity')
plt.scatter(merged[merged['predictions']==1].LON, merged[merged['predictions']==1].LAT, label='Onlap')
plt.scatter(merged[merged['predictions']==2].LON, merged[merged['predictions']==2].LAT, label='Horizontally Stratified')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))